In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
import numpy as np
max_long = 25

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [23]:
!pip install unidecode

In [24]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [25]:
import token_glove
token_glove.create_list, token_glove.token_ize, token_glove.voc_token, token_glove.dict_token, token_glove.token_tweet, token_glove.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [26]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [27]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [28]:
df = data.get_data(addon=True)
cleaned_df = data.clean_data(df)

In [29]:
cleaned_df

,id,text,target
0,1,deed reason earthquake may allah forgive u,1
1,4,forest fire near la canada,1
2,5,resident asked shelter place notified officer ...,1
3,6,people receive wildfire evacuation order calif...,1
4,7,got sent photo alaska smoke wildfire school,1
...,...,...,...
11365,11365,medium warned u well advance wrecked whole nig...,0
11366,11366,feel directly attacked consider moonbin amp ji...,0
11367,11367,feel directly attacked consider moonbin amp ji...,0
11368,11368,ok remember outcast nd au au wrecked nerve nd ...,0


In [30]:
text = token_glove.create_list(cleaned_df['text'])
token = token_glove.token_ize(text)
X = token_glove.token_tweet(text, token)

In [31]:
word_vector_matrix = token_glove.glove_vector(token)

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [34]:
y = cleaned_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.3, stratify = y)

# **CNN**

In [35]:
model = Sequential()

es = EarlyStopping(patience=10, restore_best_weights= True)

model.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model.add(Conv1D(64, 8, activation ='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'binary_crossentropy', metrics = [metrics.f1_m])
history = model.fit(X_train, y_train, batch_size=64, epochs = 400, validation_data = (X_test, y_test), callbacks = [es])

Epoch 1/400
208/208 [==============================] - 16s 71ms/step - loss: 0.6204 - f1_m: 0.1375 - val_loss: 0.5798 - val_f1_m: 0.1179
Epoch 2/400
208/208 [==============================] - 15s 73ms/step - loss: 0.5498 - f1_m: 0.2157 - val_loss: 0.5163 - val_f1_m: 0.5035
Epoch 3/400
208/208 [==============================] - 15s 70ms/step - loss: 0.5023 - f1_m: 0.4291 - val_loss: 0.4758 - val_f1_m: 0.6079
Epoch 4/400
208/208 [==============================] - 14s 67ms/step - loss: 0.4724 - f1_m: 0.5226 - val_loss: 0.4549 - val_f1_m: 0.6273
Epoch 5/400
208/208 [==============================] - 15s 71ms/step - loss: 0.4545 - f1_m: 0.5530 - val_loss: 0.4456 - val_f1_m: 0.6455
Epoch 6/400
208/208 [==============================] - 15s 74ms/step - loss: 0.4367 - f1_m: 0.5910 - val_loss: 0.4376 - val_f1_m: 0.6485
Epoch 7/400
208/208 [==============================] - 14s 69ms/step - loss: 0.4242 - f1_m: 0.6077 - val_loss: 0.4340 - val_f1_m: 0.6641
Epoch 8/400
208/208 [====================